In [2]:
import pandas as pd

# 1. Load the Datasets and Filter only the Initial Title, Body and Tags

In [ ]:
# Step 1: Load the data
df = pd.read_csv('./data/processed/Post_Combined.csv')

# Step 2: Filter only the Initial Title, Body, Tags, Comments and Answers
filter = df['Name'].isin(['Initial Title', 'Initial Body', 'Initial Tags', 'Comments', 'Answers'])
df = df[filter]

# Debug
print(df)

       PostId         CreationDate           Name  Score  \
0       90178  2008-09-18 05:06:17  Initial Title    NaN   
1       90178  2008-09-18 05:06:17   Initial Body    NaN   
2       90178  2008-09-18 05:06:17   Initial Tags    NaN   
4       90178  2008-09-18 06:01:58        Answers   19.0   
7       90178  2008-09-18 08:16:50        Answers  294.0   
..        ...                  ...            ...    ...   
463  10123953  2022-10-05 13:22:03        Answers   15.0   
464  10123953  2022-10-08 12:28:08       Comments    0.0   
465  10123953  2023-06-26 14:05:09       Comments    0.0   
466  10123953  2023-08-11 11:19:43        Answers    1.0   
467  10123953  2023-10-27 05:01:19        Answers    5.0   

                                                  Text  
0           Make a div fill the remaining screen space  
1    I am currently working on a web application, w...  
2                                   <table><css><html>  
4    <pre><code>&lt;!DOCTYPE html PUBLIC "-//W3C//D

## Grouped by PostID and Extract each row Initial version

In [ ]:
# Step 3: Group by PostId
grouped = df.groupby('PostId')

# # Debug: Display the dataframe after group by PostID object
# for post_id, group in grouped:
#     print(f"Post ID: {post_id}")
#     print(group)
#     print("-" * 40)
    
# Step 4: For each PostId, extract Title, Body, and Tags
records = []

for post_id, group in grouped:
    # Dictionary for each row. Has PostID, Initial Title, Initial BOdy,Initial Tag
    post = {
        'PostId': post_id,
        'Title': None,
        'Body': None,
        'Tags': None,
        'Comments': [],
        'Answers': []
    }
    
    # Create List for Comments and Answers so that the 'post' Dictionary could include List
    comments = []
    answers = []

    for row in group.itertuples():
        if row.Name == 'Initial Title':
            post['Title'] = row.Text
        elif row.Name == 'Initial Body':
            post['Body'] = row.Text
        elif row.Name == 'Initial Tags':
            post['Tags'] = row.Text
        elif row.Name == 'Comments':
            comments.append((row.Score, row.Text))
        elif row.Name == 'Answers':
            answers.append((row.Score, row.Text))
            
    # Sort and store top 3 by score
    post['Comments'] = [text for score, text in sorted(comments, reverse=True)[:3]]
    post['Answers'] = [text for score, text in sorted(answers, reverse=True)[:3]]
    
    records.append(post)
    
# Debug
#print(records[0])

{'PostId': 90178, 'Title': 'Make a div fill the remaining screen space', 'Body': 'I am currently working on a web application, where I want the content to fill the entire screen. I have a header `div` and a content `div`. At the moment I am using a table for the layout like so:\r\n\r\nCSS:\r\n\r\n    #page {\r\n      height: 100%;\r\n    }\r\n    #tdcontent {\r\n      height: 100%;\r\n    }\r\n\r\nHTML:\r\n\r\n    <table id="page">\r\n      <tr><td id="tdheader">\r\n        <div id="header">...</div>\r\n      </td></tr>\r\n      <tr><td id="tdcontent>\r\n        <div id="content">...</div>\r\n      </td>\r\n    </table>\r\n\r\nWhich works how I like. The table remains at 100%, and the `tdcontent` element is always less than 100%. Is there a way to achieve the same effect without using the `table`?', 'Tags': '<table><css><html>', 'Answers': '<p>max-height: -webkit-fill-available;</p>\n<p>that\'s it. all you need.</p>\n<p>On Safari you may notice that the height being incorrect when you 

In [ ]:
# Step 5: Turn records List into DataFrame
final_df = pd.DataFrame(records)

# Debug
from IPython.display import display
display(final_df)

# Step 6: Export to csv file
final_df.to_csv(path_or_buf='./data/processed/PostHistoryFirstVersion.csv', index = False) # False index param means no index for export data